# Queue & Stack

In [2]:
class Queue:
  def __init__(self):
    self.data = []

  def is_empty(self):
    return len(self.data) == 0

  def enqueue(self, value):
    self.data.append(value)

  def dequeue(self):
    return self.data.pop(0)

  def __str__(self):
    return str(self.data)


queue = Queue()
queue.enqueue(10)
queue.enqueue(20)
queue.enqueue(30)

print(queue.dequeue())
print(queue.dequeue())
queue.enqueue(40)
print(queue.dequeue())
print(queue)

10
20
30
[40]


In [3]:
class Stack:
  def __init__(self):
    self.data = []

  def is_empty(self):
    return len(self.data) == 0

  def push(self, value):
    self.data.append(value)

  def pop(self):
    return self.data.pop()

  def __str__(self):
    return str(self.data)


stack = Stack()
stack.push(10)
stack.push(20)
stack.push(30)

print(stack.pop())
print(stack.pop())
stack.push(40)
print(stack.pop())
print(stack)

30
20
40
[10]


# Convert adjacency matrix to adjacency list

In [4]:
def adjacency_matrix_to_list(adjacency_matrix):
  n = len(adjacency_matrix)
  adjacency_list = {i: [] for i in range(n)}

  for i in range(n):
    for j in range(n):
      if adjacency_matrix[i][j] == 1:
        adjacency_list[i].append(j)
  return adjacency_list


adjacency_matrix = [[0, 1, 1, 0, 0, 0, 0],
                    [1, 0, 1, 1, 1, 1, 0],
                    [1, 1, 0, 1, 1, 1, 0],
                    [0, 1, 1, 0, 1, 0, 0],
                    [0, 1, 1, 1, 0, 1, 0],
                    [0, 1, 1, 0, 1, 0, 1],
                    [0, 0, 0, 0, 0, 1, 0]]

adjacency_matrix_to_list(adjacency_matrix)

{0: [1, 2],
 1: [0, 2, 3, 4, 5],
 2: [0, 1, 3, 4, 5],
 3: [1, 2, 4],
 4: [1, 2, 3, 5],
 5: [1, 2, 4, 6],
 6: [5]}

# GrPA 1

### Walkthrough

1. **Convert the matrix**
   - Change the adjacency matrix into an adjacency list for easier traversal.

2. **BFS**
   - Use BFS to explore each person level by level, starting from Px.

3. **Track levels**
   - Track the number of steps (levels) it takes to reach each person from Px.

4. **Return connectivity level**
   - Return the connection level between Px and Py.

In [5]:
def parse_my_input(my_input):
  data = my_input.strip().split('\n')

  n = int(data[0])
  adjacency_matrix = [list(map(int, data[i+1].split())) for i in range(n)]
  px = int(data[-2])
  py = int(data[-1])

  return n, adjacency_matrix, px, py

In [6]:
def bfs(adjacency_list, start_vertex):
  visited = {vertex: False for vertex in adjacency_list}
  level = {vertex: 0 for vertex in adjacency_list}

  queue = Queue()
  queue.enqueue(start_vertex)
  visited[start_vertex] = True

  while not queue.is_empty():
    curr_vertex = queue.dequeue()

    for adj_vertex in adjacency_list[curr_vertex]:
      if not visited[adj_vertex]:
        queue.enqueue(adj_vertex)
        visited[adj_vertex] = True
        level[adj_vertex] = level[curr_vertex]+1
  return level


def find_connection_level(n, adjacency_matrix, px, py):
  adjacency_list = adjacency_matrix_to_list(adjacency_matrix)  # helper function
  level = bfs(adjacency_list, px)  # helper function
  return level[py]


find_connection_level(*parse_my_input("""
7
0 1 1 0 0 0 0
1 0 1 1 1 1 0
1 1 0 1 1 1 0
0 1 1 0 1 0 0
0 1 1 1 0 1 0
0 1 1 0 1 0 1
0 0 0 0 0 1 0
6
0
"""))

3

# GrPA 2

### Walkthrough

1. **BFS**
   - Implement BFS to explore the network starting from a given tank and mark all reachable tanks as visited.

2. **Make adjacency list**
   - Convert the list of pipes into an adjacency list for easier traversal.

3. **Check each tank**
   - For each tank, run BFS to see if all other tanks are reachable from it.

4. **Return master tank**
   - If a tank can reach all other tanks, return that tank as the master tank. If no such tank exists, return 0.

In [7]:
def parse_my_input(my_input):
  data = my_input.strip().split('\n')

  V = list(map(int, data[0].split()))
  E = [list(map(int, edge.split())) for edge in data[2:]]

  return V, E

In [8]:
def bfs(adjacency_list, start_vertex):
  visited = {vertex: False for vertex in adjacency_list}

  queue = Queue()
  queue.enqueue(start_vertex)
  visited[start_vertex] = True

  while not queue.is_empty():
    curr_vertex = queue.dequeue()

    for adj_vertex in adjacency_list[curr_vertex]:
      if not visited[adj_vertex]:
        queue.enqueue(adj_vertex)
        visited[adj_vertex] = True
  return visited


def find_master_tank(tanks, pipes):
  adjacency_list = {u: [] for u in tanks}
  for u, v in pipes:
    adjacency_list[u].append(v)

  for vertex in adjacency_list:
    visited = bfs(adjacency_list, vertex)  # helper function
    if all(visited.values()):
      return vertex
  return 0


find_master_tank(*parse_my_input("""
1 2 3 4
3
1 2
2 3
2 4
"""))

1

# GrPA 3

### Walkthrough

1. **DFS and Topological Sort**
   - Perform DFS to generate a topological order of the graph using the `topological_sort` function.

2. **Longest path calculation**
   - Initialize longest path lengths to -1 and set the start node's length to 0.
   - Maintain a predecessor dictionary to trace paths.

3. **Path reconstruction**
   - Identify the node with the maximum path length.
   - Reconstruct the path from this node back to the start using the predecessor dictionary.

In [9]:
def dfs(adjacency_list, curr_vertex, visited, stack):
  visited[curr_vertex] = True

  for neighbor in adjacency_list[curr_vertex]:
    if not visited[neighbor]:
      dfs(adjacency_list, neighbor, visited, stack)

  stack.append(curr_vertex)


def topological_sort(adjacency_list):
  visited = {vertex: False for vertex in adjacency_list}
  stack = []

  for vertex in adjacency_list:
    if not visited[vertex]:
      dfs(adjacency_list, vertex, visited, stack)

  # the stack contains the topologically sorted order in reverse
  return stack[::-1]

In [10]:
def longest_path(adjacency_list):
  # compute topological order
  topological_order = topological_sort(adjacency_list)

  # initialize distances and predecessors
  distances = {vertex: -1 for vertex in adjacency_list}
  predecessor = {vertex: None for vertex in adjacency_list}

  # start from the first node in topological order
  start_vertex = topological_order[0]
  distances[start_vertex] = 0

  # compute distances, predecessor
  for parent in topological_order:
    if distances[parent] != -1:  # if parent has a distance assigned
      for child in adjacency_list[parent]:
        new_distance = distances[parent] + 1
        if new_distance > distances[child]:
          distances[child] = new_distance
          predecessor[child] = parent

  return distances, predecessor


def display_path_info(distances, predecessor):
  # display information for visualization
  sorted_vertices = sorted(distances, key=distances.get)
  display([(vertex, distances[vertex], predecessor[vertex]) for vertex in sorted_vertices])


def reconstruct_path(last_vertex, predecessor):
  # reconstruct the path from last_vertex back to start_vertex
  path = []
  while last_vertex is not None:
    path.append(last_vertex)
    last_vertex = predecessor[last_vertex]
  path.reverse()
  return path


def long_journey(adjacency_list):
  distances, predecessor = longest_path(adjacency_list)

  # visualize distances and predecessors
  display_path_info(distances, predecessor)

  # reconstruct the path
  last_vertex = max(distances, key=distances.get)
  return reconstruct_path(last_vertex, predecessor)

In [11]:
adjacency_list = {
    'Madurai': ['Cochin', 'Kanyakumari'],
    'Vaishali': [],
    'Varanasi': ['Khajuraho', 'Bodhgaya'],
    'Thiruvanandhapuram': ['Kanyakumari'],
    'Udaipur': ['Gir', 'Ajanta'],
    'Rishikesh': ['Delhi'],
    'Shimla': ['Rishikesh'],
    'Bangalore': ['Chennai', 'Madurai'],
    'Agra': ['Ranthambore'],
    'Ellora': ['Aurangabad'],
    'Bodhgaya': ['Kolkatta'],
    'Cochin': ['Thiruvanandhapuram'],
    'Pushkar': ['Udaipur', 'Ranthambore'],
    'Ranthambore': ['Khajuraho'],
    'Gir': [],
    'Aurangabad': ['Mumbai'],
    'Kolkatta': ['Ajanta', 'Bangalore', 'Chennai'],
    'Chennai': ['Madurai'],
    'Sravasti': ['Kushinagar'],
    'Leh': ['Shimla'],
    'Sarnath': ['Varanasi'],
    'Delhi': ['Jaipur', 'Agra', 'Sravasti'],
    'Goa': ['Cochin', 'Bangalore'],
    'Kanyakumari': [],
    'Kushinagar': ['Sarnath', 'Vaishali'],
    'Khajuraho': ['Ajanta'],
    'Jaipur': ['Pushkar'],
    'Mumbai': ['Goa'],
    'Ajanta': ['Ellora', 'Aurangabad']
}

expected_output = ['Leh', 'Shimla', 'Rishikesh', 'Delhi', 'Sravasti', 'Kushinagar', 'Sarnath', 'Varanasi', 'Bodhgaya', 'Kolkatta',
                   'Ajanta', 'Ellora', 'Aurangabad', 'Mumbai', 'Goa', 'Bangalore', 'Chennai', 'Madurai', 'Cochin', 'Thiruvanandhapuram', 'Kanyakumari']

long_journey(adjacency_list) == expected_output

[('Leh', 0, None),
 ('Shimla', 1, 'Leh'),
 ('Rishikesh', 2, 'Shimla'),
 ('Delhi', 3, 'Rishikesh'),
 ('Agra', 4, 'Delhi'),
 ('Sravasti', 4, 'Delhi'),
 ('Jaipur', 4, 'Delhi'),
 ('Pushkar', 5, 'Jaipur'),
 ('Kushinagar', 5, 'Sravasti'),
 ('Vaishali', 6, 'Kushinagar'),
 ('Udaipur', 6, 'Pushkar'),
 ('Ranthambore', 6, 'Pushkar'),
 ('Sarnath', 6, 'Kushinagar'),
 ('Varanasi', 7, 'Sarnath'),
 ('Gir', 7, 'Udaipur'),
 ('Bodhgaya', 8, 'Varanasi'),
 ('Khajuraho', 8, 'Varanasi'),
 ('Kolkatta', 9, 'Bodhgaya'),
 ('Ajanta', 10, 'Kolkatta'),
 ('Ellora', 11, 'Ajanta'),
 ('Aurangabad', 12, 'Ellora'),
 ('Mumbai', 13, 'Aurangabad'),
 ('Goa', 14, 'Mumbai'),
 ('Bangalore', 15, 'Goa'),
 ('Chennai', 16, 'Bangalore'),
 ('Madurai', 17, 'Chennai'),
 ('Cochin', 18, 'Madurai'),
 ('Thiruvanandhapuram', 19, 'Cochin'),
 ('Kanyakumari', 20, 'Thiruvanandhapuram')]

True

### Alternate solution, do not prefer 💀

1. **DFS for Path Exploration**:
    - The `dfs` function uses depth-first search to explore paths starting from a city.
    - It marks cities as visited and tracks the longest path found recursively.

   1. **Path Exploration**:
       - For each unvisited neighboring city, it recursively explores and updates the longest path if a longer path is found.

   2. **Backtracking**:
       - After exploring all neighbors, it backtracks by removing the city from the path and visited set to explore other paths.

2. **Longest Path Calculation**:
    - The `long_journey` function iterates through cities, initiating `dfs` from each to find the longest path.
    - It records and returns the longest path found across all starting cities.

In [12]:
def dfs(adjacency_list, city, visited, path):
  # mark the current city as visited and add to path
  visited[city] = True
  path.append(city)
  longest_path = list(path)  # track the longest path found

  # explore neighbors
  for neighbor in adjacency_list[city]:
    if not visited[neighbor]:  # if neighbor not visited
      current_path = dfs(adjacency_list, neighbor, visited, path)
      # update longest_path if current path is longer
      if len(current_path) > len(longest_path):
        longest_path = current_path

  path.pop()  # backtrack: remove current city from path
  visited[city] = False  # mark current city as not visited
  return longest_path


def longest_path(adjacency_list):
  longest_path_overall = []  # initialize overall longest path

  # find longest path starting from each city
  for city in adjacency_list:
    visited = {vertex: False for vertex in adjacency_list}  # mark all cities as not visited
    current_longest_path = dfs(adjacency_list, city, visited, [])
    # update overall longest path if current one is longer
    if len(current_longest_path) > len(longest_path_overall):
      longest_path_overall = current_longest_path

  return longest_path_overall


def long_journey(adjacency_list):
  # return longest path found
  return longest_path(adjacency_list)

In [13]:
adjacency_list = {
    'Madurai': ['Cochin', 'Kanyakumari'],
    'Vaishali': [],
    'Varanasi': ['Khajuraho', 'Bodhgaya'],
    'Thiruvanandhapuram': ['Kanyakumari'],
    'Udaipur': ['Gir', 'Ajanta'],
    'Rishikesh': ['Delhi'],
    'Shimla': ['Rishikesh'],
    'Bangalore': ['Chennai', 'Madurai'],
    'Agra': ['Ranthambore'],
    'Ellora': ['Aurangabad'],
    'Bodhgaya': ['Kolkatta'],
    'Cochin': ['Thiruvanandhapuram'],
    'Pushkar': ['Udaipur', 'Ranthambore'],
    'Ranthambore': ['Khajuraho'],
    'Gir': [],
    'Aurangabad': ['Mumbai'],
    'Kolkatta': ['Ajanta', 'Bangalore', 'Chennai'],
    'Chennai': ['Madurai'],
    'Sravasti': ['Kushinagar'],
    'Leh': ['Shimla'],
    'Sarnath': ['Varanasi'],
    'Delhi': ['Jaipur', 'Agra', 'Sravasti'],
    'Goa': ['Cochin', 'Bangalore'],
    'Kanyakumari': [],
    'Kushinagar': ['Sarnath', 'Vaishali'],
    'Khajuraho': ['Ajanta'],
    'Jaipur': ['Pushkar'],
    'Mumbai': ['Goa'],
    'Ajanta': ['Ellora', 'Aurangabad']
}

expected_output = ['Leh', 'Shimla', 'Rishikesh', 'Delhi', 'Sravasti', 'Kushinagar', 'Sarnath', 'Varanasi', 'Bodhgaya', 'Kolkatta',
                   'Ajanta', 'Ellora', 'Aurangabad', 'Mumbai', 'Goa', 'Bangalore', 'Chennai', 'Madurai', 'Cochin', 'Thiruvanandhapuram', 'Kanyakumari']

long_journey(adjacency_list) == expected_output

True